In [1]:
import pandas as pd
import numpy as np
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)

In [2]:
season_list = ['1718', '1819', '1920', '2021', '2122', '2223', '2324', '2425']
league = 'E0'

df_list = []

for season in season_list:
    data_url = f'https://www.football-data.co.uk/mmz4281/{season}/{league}.csv'
    df = pd.read_csv(data_url)
    df_list.append(df)

main_df = pd.concat(df_list)

main_df['Date'] = pd.to_datetime(main_df['Date'], format='%d/%m/%Y')
main_df = main_df.sort_values(by='Date')

df = main_df
df = df[::-1]

In [3]:
def get_teams_result(df):
    resultat = {}

    teams = set(df["HomeTeam"]).union(set(df["AwayTeam"]))

    for team in teams:
        resultat[team] = {}
        
    for _, rad in df.iterrows():
        hemma, borta = rad["HomeTeam"], rad["AwayTeam"]
        mål_a, mål_b = rad["FTHG"], rad["FTAG"]

        if hemma not in resultat:
            resultat[hemma] = {}
        if borta not in resultat:
            resultat[borta] = {}

        if borta not in resultat[hemma]:
            resultat[hemma][borta] = {"Vinster": 0, "Förluster": 0, "Oavgjorda": 0}
        if hemma not in resultat[borta]:
            resultat[borta][hemma] = {"Vinster": 0, "Förluster": 0, "Oavgjorda": 0}

        if mål_a > mål_b: 
            resultat[hemma][borta]["Vinster"] += 1
            resultat[borta][hemma]["Förluster"] += 1
        elif mål_a < mål_b: 
            resultat[borta][hemma]["Vinster"] += 1
            resultat[hemma][borta]["Förluster"] += 1
        else:
            resultat[hemma][borta]["Oavgjorda"] += 1
            resultat[borta][hemma]["Oavgjorda"] += 1
            
    return resultat

#get_teams_result(df)['Man City']

In [4]:
def get_home_vs_away_summary(df, home_team, away_team):

    matcher = df[(df["HomeTeam"] == home_team) & (df["AwayTeam"] == away_team)]

    resultat = {"Vinster": 0, "Förluster": 0, "Oavgjorda": 0}

    for _, rad in matcher.iterrows():
        mål_hemma = rad["FTHG"]
        mål_borta = rad["FTAG"]

        if mål_hemma > mål_borta:
            resultat["Vinster"] += 1
        elif mål_hemma < mål_borta:
            resultat["Förluster"] += 1
        else:
            resultat["Oavgjorda"] += 1

    return resultat

#get_home_vs_away_summary(df,'Liverpool', 'Arsenal')

In [5]:
def get_teams_by_league(league):
    league_df = df[df["Div"] == league]
    home_teams = league_df['HomeTeam'].astype(str).tolist()
    away_teams = league_df['AwayTeam'].astype(str).tolist()
    unique_teams = sorted(np.unique(home_teams + away_teams))
    return unique_teams



In [6]:
col_list = ['Div', 'Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'B365H', 'B365D', 'B365A']

In [ ]:
def get_team_form(df, team, num_matches=20):

    team_matches = df[(df['HomeTeam'] == team) | (df['AwayTeam'] == team)].copy()

    team_matches = team_matches.sort_values(by='Date', ascending=False).head(num_matches)

    form = []

    for _, row in team_matches.iterrows():
        is_home = row['HomeTeam'] == team
        goals_for = row['FTHG'] if is_home else row['FTAG']
        goals_against = row['FTAG'] if is_home else row['FTHG']

        if goals_for > goals_against:
            form.append('W')
        elif goals_for == goals_against:
            form.append('D')
        else:
            form.append('L')

    return form[::-1]  

#get_team_form(df, 'Tottenham')

In [8]:
bet_providers = ['B365', 'BFE', 'BF', 'PS']

home_cols = []
draw_cols = []
away_cols = []

for bet in bet_providers:
    home_cols.append(bet + 'H')
    draw_cols.append(bet + 'D')
    away_cols.append(bet + 'A')


df['home_max_odds'] = df[home_cols].max(axis=1)
df['draw_max_odds'] = df[draw_cols].max(axis=1)
df['away_max_odds'] = df[away_cols].max(axis=1)

col_list = ['Div', 'Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'B365H', 'B365D', 'B365A', 'PSH', 'PSD', 'PSA', 'home_max_odds', 'draw_max_odds', 'away_max_odds']
df = df[col_list]

/var/folders/rt/kk5y63zd2dl20zp4x97pdp2m0000gn/T/ipykernel_75396/1742109355.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['home_max_odds'] = df[home_cols].max(axis=1)
/var/folders/rt/kk5y63zd2dl20zp4x97pdp2m0000gn/T/ipykernel_75396/1742109355.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['home_max_odds'] = df[home_cols].max(axis=1)
/var/folders/rt/kk5y63zd2dl20zp4x97pdp2m0000gn/T/ipykernel_75396/1742109355.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of cal

In [9]:
def divide(df):
    res = 1/df
    return res


test = df['B365A']


for index, odd in enumerate(df['PSH']):
    df.at[index, 'home_probability'] = divide(odd)
for index, odd in enumerate(df['PSD']):
    df.at[index, 'draw_probability'] = divide(odd)
for index, odd in enumerate(df['PSA']):
    df.at[index, 'away_probability'] = divide(odd)

tot_prob = df['home_probability'] + df['draw_probability'] + df['away_probability']
print(tot_prob)

319     1.030487
318     1.033956
317     1.037391
314     1.039751
316     1.032722
          ...   
8175         NaN
8176         NaN
8177         NaN
8178         NaN
8179         NaN
Length: 10780, dtype: float64
